In [53]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import time
from PIL import Image
from sklearn import svm
from sklearn.model_selection import train_test_split

In [111]:
# def load_dataset(verbose=False):
#     rootDir = 'Images'
#     images = np.zeros((1, 1280), dtype=np.uint8)
#     labels = np.empty(20580, dtype=object)
#     i = 0
#     j = 0
#     sift = cv2.SIFT_create(10)
#     for dirName, subdirList, fileList in os.walk(rootDir):
#         #print('Found directory: %s' % dirName)
#         if dirName != 'Images':
#             l = dirName.split('-')
#             label = l[1]
# #             if j == 5:
# #                 break
#             j += 1
#             print(label, j)
#         count = 0
#         for fname in fileList:
#             if dirName != 'Images':
#                 img = cv2.imread(dirName + '/' + fname, 1)
#                 file = open("Annotation\\" + dirName.replace('Images\\', '') + "\\" + fname.replace('.jpg', '')).read()
#                 bnd = [s.strip() for s in re.findall(r'<bndbox>(.*)</bndbox>', file, re.DOTALL)][0].replace('<', ' ').replace('>', ' ').split()
#                 bndbox = [bnd[1], bnd[4], bnd[7], bnd[10]]
#                 im3 = img[int(bndbox[1]):int(bndbox[3]), int(bndbox[0]):int(bndbox[2])]
#                 im = Image.fromarray(im3)
#                 im.save(dirName + '/' + fname)

In [112]:
# load_dataset()

Chihuahua 1
Japanese_spaniel 2
Maltese_dog 3
Pekinese 4
Shih_Tzu 5
Blenheim_spaniel 6
papillon 7
toy_terrier 8
Rhodesian_ridgeback 9
Afghan_hound 10
basset 11
beagle 12
bloodhound 13
bluetick 14
black_and_tan_coonhound 15
Walker_hound 16
English_foxhound 17
redbone 18
borzoi 19
Irish_wolfhound 20
Italian_greyhound 21
whippet 22
Ibizan_hound 23
Norwegian_elkhound 24
otterhound 25
Saluki 26
Scottish_deerhound 27
Weimaraner 28
Staffordshire_bullterrier 29
American_Staffordshire_terrier 30
Bedlington_terrier 31
Border_terrier 32
Kerry_blue_terrier 33
Irish_terrier 34
Norfolk_terrier 35
Norwich_terrier 36
Yorkshire_terrier 37
wire_haired_fox_terrier 38
Lakeland_terrier 39
Sealyham_terrier 40
Airedale 41
cairn 42
Australian_terrier 43
Dandie_Dinmont 44
Boston_bull 45
miniature_schnauzer 46
giant_schnauzer 47
standard_schnauzer 48
Scotch_terrier 49
Tibetan_terrier 50
silky_terrier 51
soft_coated_wheaten_terrier 52
West_Highland_white_terrier 53
Lhasa 54
flat_coated_retriever 55
curly_coated_r

In [113]:
from scipy.io import loadmat
mat = loadmat('train_data.mat')
train_data = mat['train_data']
train_labels = mat['train_info'][0, 0]['labels'].reshape(12000, )
print(train_data.shape, train_labels.shape)

(12000, 12000) (12000,)


In [ ]:
clf = svm.SVC(kernel='precomputed')
clf.fit(train_data, train_labels)

In [ ]:
mat1 = loadmat('test_data.mat')
test_data = mat1['test_data']
test_labels = mat1['test_info'][0, 0]['labels'].reshape(len(test_data), )
print(test_data.shape, test_labels.shape)

In [ ]:
preds = clf.predict(test_data)
print(preds)
print(test_labels)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels, preds)
np.savetxt('confusion.txt', cm)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, preds)

In [ ]:
'''

CNN BELOW

'''

In [89]:
import torch
import torchvision
import torchvision.transforms as transforms
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [90]:
trans = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
data = torchvision.datasets.ImageFolder(root='Images', transform=trans)
traindata, testdata = torch.utils.data.random_split(data, [12000, 8580])
print(traindata, testdata)

<torch.utils.data.dataset.Subset object at 0x000001F798B937C0> <torch.utils.data.dataset.Subset object at 0x000001F798B93130>


In [91]:
trainloader = torch.utils.data.DataLoader(traindata, batch_size=4)
testloader = torch.utils.data.DataLoader(testdata, batch_size = 4)
print(trainloader, testloader)

<torch.utils.data.dataloader.DataLoader object at 0x000001F83C6DB580> <torch.utils.data.dataloader.DataLoader object at 0x000001F78376AEE0>


In [92]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 60, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(60, 16, 5)
        self.fc1 = nn.Linear(16 * 29*29, 240)
        self.fc2 = nn.Linear(240, 120)
#         self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), 16*29*29)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        return x


net = Net()

In [93]:
net.to(device)

Net(
  (conv1): Conv2d(3, 60, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(60, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=13456, out_features=240, bias=True)
  (fc2): Linear(in_features=240, out_features=120, bias=True)
)

In [94]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [95]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  1000] loss: 2.337
[1,  2000] loss: 2.216
[1,  3000] loss: 2.173
Finished Training


In [96]:
PATH = './model.pth'
torch.save(net.state_dict(), PATH)

In [46]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [97]:
correct = 0.0
total = 0.0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 8580 test images: %.3f %%' % (
    100 * correct / total))

Accuracy of the network on the 8580 test images: 5.058 %


In [86]:
f = open("classes.txt", "r")
classmap = {}
for i in range(120):
    classs = f.readline().split(" ")
    classmap[i] = classs[0]

In [98]:
class_correct = list(0. for i in range(120))
class_total = list(0. for i in range(120))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(120):
    print('Accuracy of %30s : %10d %%' % (
        classmap[i], 100 * class_correct[i] / class_total[i]))

Accuracy of                      Chihuahua :          0 %
Accuracy of               Japanese_spaniel :          0 %
Accuracy of                    Maltese_dog :          8 %
Accuracy of                       Pekinese :          0 %
Accuracy of                       Shih_Tzu :          8 %
Accuracy of               Blenheim_spaniel :          0 %
Accuracy of                       papillon :         46 %
Accuracy of                    toy_terrier :          0 %
Accuracy of            Rhodesian_ridgeback :          0 %
Accuracy of                   Afghan_hound :          0 %
Accuracy of                         basset :          0 %
Accuracy of                         beagle :          5 %
Accuracy of                     bloodhound :          2 %
Accuracy of                       bluetick :          0 %
Accuracy of        black_and_tan_coonhound :         11 %
Accuracy of                   Walker_hound :          0 %
Accuracy of               English_foxhound :          0 %
Accuracy of   